# Generate Contigs from a Collection of Reads

In [133]:
def read_file(filename):
    return [line.rstrip('\n') for line in open(filename)]

def write_answer(answ):    
    fld = open("output.txt", "w")
    fld.write(" ".join([walk for walk in answ]))
    fld.close()

def de_bruijn_graph(kmers):
    arcs = {}
    for kmer in kmers:
        if kmer[:-1] not in list(arcs.keys()):
            arcs.update({kmer[:-1]: [kmer[1:]]})
        else:
            arcs[kmer[:-1]].append(kmer[1:])            
    return arcs

def find_nodes(graph):
    deg = {}
    [deg.update({node: [0, len(graph[node])]}) for node in graph.keys()]
    for in_node in graph.keys():
        for out_node in graph[in_node]:
            if out_node in deg.keys():
                deg[out_node][0] += 1
    return [key for key in deg.keys() if deg[key] != [1, 1] ]

def find_startnode(graph, join_nodes):
    return [node for node in graph.keys() if node in join_nodes][0] 

def walk_until_branch(graph, join_nodes):
    walking = [find_startnode(graph, join_nodes)]

    while((walking[-1] in graph.keys() and walking[-1] not in join_nodes) or len(walking) == 1):
        walking.append(graph[walking[-1]].pop())
        if graph[walking[-2]] == []:
            graph.pop(walking[-2])
    return walking

def non_branching_walks(graph):
    join_nodes = find_nodes(graph)
    walks = []
    while(graph != {}):
        walks.append(walk_until_branch(graph, join_nodes))
    return walks

def reconstruct_dna(path):
    return ''.join([dna[0] for dna in path])+path[-1][1:]

In [134]:
reads = read_file("rosalind_ba3k.txt")
graph = de_bruijn_graph(reads)
walks = non_branching_walks(graph)
write_answer([reconstruct_dna(walk) for walk in walks])